<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#This-nootebooks-is-used-to-compute-the-p-values-used-in-the-Scalp_Plots_Pvals-notebook.-It-should-be-transformed-into-a-.py-file-and-contain-well-organized-functions." data-toc-modified-id="This-nootebooks-is-used-to-compute-the-p-values-used-in-the-Scalp_Plots_Pvals-notebook.-It-should-be-transformed-into-a-.py-file-and-contain-well-organized-functions.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>This nootebooks is used to compute the p-values used in the Scalp_Plots_Pvals notebook. It should be transformed into a .py file and contain well organized functions.</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#In-order-to-visualize-EEG-distribution,-scalp-plots-are-created-using-hytools" data-toc-modified-id="In-order-to-visualize-EEG-distribution,-scalp-plots-are-created-using-hytools-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>In order to visualize EEG distribution, scalp plots are created using hytools</a></span></li></ul></li></ul></li><li><span><a href="#COMPUTE-P-VALUES-FOR-GENDER-DIFFERENCES." data-toc-modified-id="COMPUTE-P-VALUES-FOR-GENDER-DIFFERENCES.-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>COMPUTE P-VALUES FOR GENDER DIFFERENCES.</a></span><ul class="toc-item"><li><span><a href="#Get-DELTA-p-values" data-toc-modified-id="Get-DELTA-p-values-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Get DELTA p values</a></span></li><li><span><a href="#Get-THETA-p-values" data-toc-modified-id="Get-THETA-p-values-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Get THETA p-values</a></span></li><li><span><a href="#Get-ALPHA-p-values" data-toc-modified-id="Get-ALPHA-p-values-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Get ALPHA p-values</a></span></li><li><span><a href="#Get-BETA-p-values" data-toc-modified-id="Get-BETA-p-values-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Get BETA p-values</a></span></li></ul></li><li><span><a href="#COMPUTING-P-VALUES-FOR-DIFFERENCES-IN-ADHD-SUBTYPE" data-toc-modified-id="COMPUTING-P-VALUES-FOR-DIFFERENCES-IN-ADHD-SUBTYPE-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>COMPUTING P-VALUES FOR DIFFERENCES IN ADHD SUBTYPE</a></span><ul class="toc-item"><li><span><a href="#Get-DELTA-p-values" data-toc-modified-id="Get-DELTA-p-values-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Get DELTA p values</a></span></li><li><span><a href="#Get-THETA-p-values" data-toc-modified-id="Get-THETA-p-values-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Get THETA p values</a></span></li><li><span><a href="#Get-ALPHA-p-values" data-toc-modified-id="Get-ALPHA-p-values-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Get ALPHA p values</a></span></li><li><span><a href="#Get-BETA-p-values" data-toc-modified-id="Get-BETA-p-values-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Get BETA p values</a></span></li></ul></li></ul></div>

# This nootebooks is used to compute the p-values used in the Scalp_Plots_Pvals notebook. It should be transformed into a .py file and contain well organized functions.

In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
import seaborn as sns
import os
from statsmodels.sandbox.stats.multicomp import multipletests
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_eeg = pd.read_csv('Data/df_eeg_topo', index_col=[0])
#gender_pvals = pd.read_csv('Data/pvals', index_col=[0])


In [3]:
# getting fft_abs_power for each waveband separately 

alpha = df_eeg.loc[df_eeg['brain_oscillation'] == 'Alpha']
alpha = alpha[['electrode','fft_abs_power']]
beta = df_eeg.loc[df_eeg['brain_oscillation'] == 'Beta']
beta = beta[['electrode','fft_abs_power']]
delta = df_eeg.loc[df_eeg['brain_oscillation'] == 'Delta']
delta = delta[['electrode','fft_abs_power']]
theta = df_eeg.loc[df_eeg['brain_oscillation'] == 'Theta']
theta = theta[['electrode','fft_abs_power']]


In [4]:
avg_abspow_by_electro_alpha = alpha.groupby('electrode')['fft_abs_power'].agg('mean')
avg_abspow_by_electro_beta = beta.groupby('electrode')['fft_abs_power'].agg('mean')
avg_abspow_by_electro_delta = delta.groupby('electrode')['fft_abs_power'].agg('mean')
avg_abspow_by_electro_theta = theta.groupby('electrode')['fft_abs_power'].agg('mean')


In [5]:
df_analysis = pd.read_csv('Data/df_analysis', index_col=[0])
df_analysis

,id,Gender,subtype,adhdtype,brain_oscillation,electrode,fft_abs_power,cIM,cHR,cIE,cSC,Aqtot,Aqaudi,Aqvis,RCQtot,RCQaudi,RCQvis
0,1,1,mixed,2,Delta,FP1,15.376173,17.0,31.0,29.0,9.0,90.0,91.0,92.0,94.0,80.0,110.0
133,1,1,mixed,2,Delta,F7,7.968847,17.0,31.0,29.0,9.0,90.0,91.0,92.0,94.0,80.0,110.0
134,1,1,mixed,2,Delta,F3,9.749313,17.0,31.0,29.0,9.0,90.0,91.0,92.0,94.0,80.0,110.0
135,1,1,mixed,2,Delta,Fz,10.845292,17.0,31.0,29.0,9.0,90.0,91.0,92.0,94.0,80.0,110.0
136,1,1,mixed,2,Delta,F4,10.679267,17.0,31.0,29.0,9.0,90.0,91.0,92.0,94.0,80.0,110.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15958,702,1,inat,1,Gamma,C4,1.198341,17.0,14.0,10.0,9.0,91.0,88.0,95.0,94.0,87.0,104.0
15959,702,1,inat,1,Beta2,C3,2.209228,17.0,14.0,10.0,9.0,91.0,88.0,95.0,94.0,87.0,104.0
15960,702,1,inat,1,Beta1,Pz,12.479250,17.0,14.0,10.0,9.0,91.0,88.0,95.0,94.0,87.0,104.0
15935,702,1,inat,1,Beta1,C4,5.618459,17.0,14.0,10.0,9.0,91.0,88.0,95.0,94.0,87.0,104.0


In [6]:
df_analysis = pd.read_csv('Data/df_analysis', index_col=[0])
from sklearn.preprocessing import LabelEncoder



df_analysis = df_analysis[['Gender', 'adhdtype','brain_oscillation', 'electrode', 'fft_abs_power']]

df_Delta  = df_analysis.loc[df_analysis['brain_oscillation'] == 'Delta']
df_Theta  = df_analysis.loc[df_analysis['brain_oscillation'] == 'Theta']
df_Alpha  = df_analysis.loc[df_analysis['brain_oscillation'] == 'Alpha']
df_Beta  = df_analysis.loc[df_analysis['brain_oscillation'] == 'Beta']

#saving those new waveband dfs info csv files
df_Delta.to_csv('Data/df_Delta')
df_Theta.to_csv('Data/df_Theta')
df_Alpha.to_csv('Data/df_Alpha')
df_Beta.to_csv('Data/df_Beta')



In [7]:
df_Delta_women = df_Delta.loc[df_Delta['Gender'] == 1]
df_Delta_men = df_Delta.loc[df_Delta['Gender'] == 2]

df_Theta_women = df_Theta.loc[df_Theta['Gender'] == 1]
df_Theta_men = df_Theta.loc[df_Theta['Gender'] == 2]

df_Alpha_women = df_Alpha.loc[df_Alpha['Gender'] == 1]
df_Alpha_men = df_Alpha.loc[df_Alpha['Gender'] == 2]


df_Beta_women = df_Beta.loc[df_Beta['Gender'] == 1]
df_Beta_men = df_Beta.loc[df_Beta['Gender'] == 2]
df_Delta_women

,Gender,adhdtype,brain_oscillation,electrode,fft_abs_power
0,1,2,Delta,FP1,15.376173
133,1,2,Delta,F7,7.968847
134,1,2,Delta,F3,9.749313
135,1,2,Delta,Fz,10.845292
136,1,2,Delta,F4,10.679267
...,...,...,...,...,...
16032,1,1,Delta,P4,20.394968
16034,1,1,Delta,O1,18.520756
16035,1,1,Delta,O2,17.845653
16017,1,1,Delta,FP1,19.854420


In [8]:
Delta_C3 = df_Delta.loc[df_Delta['electrode'] == 'C3']
Delta_C4 = df_Delta.loc[df_Delta['electrode'] == 'C4']
Delta_Cz = df_Delta.loc[df_Delta['electrode'] == 'Cz']
Delta_F3 = df_Delta.loc[df_Delta['electrode'] == 'F3']
Delta_F4 = df_Delta.loc[df_Delta['electrode'] == 'F4']
Delta_F7 = df_Delta.loc[df_Delta['electrode'] == 'F7']
Delta_F8 = df_Delta.loc[df_Delta['electrode'] == 'F8']
Delta_FP1 = df_Delta.loc[df_Delta['electrode'] == 'FP1']
Delta_FP2 = df_Delta.loc[df_Delta['electrode'] == 'FP2']
Delta_Fz = df_Delta.loc[df_Delta['electrode'] == 'Fz']
Delta_O1 = df_Delta.loc[df_Delta['electrode'] == 'O1']
Delta_O2 = df_Delta.loc[df_Delta['electrode'] == 'O2']
Delta_P3 = df_Delta.loc[df_Delta['electrode'] == 'P3']
Delta_P4 = df_Delta.loc[df_Delta['electrode'] == 'P4']
Delta_Pz = df_Delta.loc[df_Delta['electrode'] == 'Pz']
Delta_T3 = df_Delta.loc[df_Delta['electrode'] == 'T3']
Delta_T4 = df_Delta.loc[df_Delta['electrode'] == 'T4']
Delta_T5 = df_Delta.loc[df_Delta['electrode'] == 'T5']
Delta_T6 = df_Delta.loc[df_Delta['electrode'] == 'T6']

Theta_C3 = df_Theta.loc[df_Theta['electrode'] == 'C3']
Theta_C4 = df_Theta.loc[df_Theta['electrode'] == 'C4']
Theta_Cz = df_Theta.loc[df_Theta['electrode'] == 'Cz']
Theta_F3 = df_Theta.loc[df_Theta['electrode'] == 'F3']
Theta_F4 = df_Theta.loc[df_Theta['electrode'] == 'F4']
Theta_F7 = df_Theta.loc[df_Theta['electrode'] == 'F7']
Theta_F8 = df_Theta.loc[df_Theta['electrode'] == 'F8']
Theta_FP1 = df_Theta.loc[df_Theta['electrode'] == 'FP1']
Theta_FP2 = df_Theta.loc[df_Theta['electrode'] == 'FP2']
Theta_Fz = df_Theta.loc[df_Theta['electrode'] == 'Fz']
Theta_O1 = df_Theta.loc[df_Theta['electrode'] == 'O1']
Theta_O2 = df_Theta.loc[df_Theta['electrode'] == 'O2']
Theta_P3 = df_Theta.loc[df_Theta['electrode'] == 'P3']
Theta_P4 = df_Theta.loc[df_Theta['electrode'] == 'P4']
Theta_Pz = df_Theta.loc[df_Theta['electrode'] == 'Pz']
Theta_T3 = df_Theta.loc[df_Theta['electrode'] == 'T3']
Theta_T4 = df_Theta.loc[df_Theta['electrode'] == 'T4']
Theta_T5 = df_Theta.loc[df_Theta['electrode'] == 'T5']
Theta_T6 = df_Theta.loc[df_Theta['electrode'] == 'T6']

Alpha_C3 = df_Alpha.loc[df_Alpha['electrode'] == 'C3']
Alpha_C4 = df_Alpha.loc[df_Alpha['electrode'] == 'C4']
Alpha_Cz = df_Alpha.loc[df_Alpha['electrode'] == 'Cz']
Alpha_F3 = df_Alpha.loc[df_Alpha['electrode'] == 'F3']
Alpha_F4 = df_Alpha.loc[df_Alpha['electrode'] == 'F4']
Alpha_F7 = df_Alpha.loc[df_Alpha['electrode'] == 'F7']
Alpha_F8 = df_Alpha.loc[df_Alpha['electrode'] == 'F8']
Alpha_FP1 = df_Alpha.loc[df_Alpha['electrode'] == 'FP1']
Alpha_FP2 = df_Alpha.loc[df_Alpha['electrode'] == 'FP2']
Alpha_Fz = df_Alpha.loc[df_Alpha['electrode'] == 'Fz']
Alpha_O1 = df_Alpha.loc[df_Alpha['electrode'] == 'O1']
Alpha_O2 = df_Alpha.loc[df_Alpha['electrode'] == 'O2']
Alpha_P3 = df_Alpha.loc[df_Alpha['electrode'] == 'P3']
Alpha_P4 = df_Alpha.loc[df_Alpha['electrode'] == 'P4']
Alpha_Pz = df_Alpha.loc[df_Alpha['electrode'] == 'Pz']
Alpha_T3 = df_Alpha.loc[df_Alpha['electrode'] == 'T3']
Alpha_T4 = df_Alpha.loc[df_Alpha['electrode'] == 'T4']
Alpha_T5 = df_Alpha.loc[df_Alpha['electrode'] == 'T5']
Alpha_T6 = df_Alpha.loc[df_Alpha['electrode'] == 'T6']

Beta_C3 = df_Beta.loc[df_Beta['electrode'] == 'C3']
Beta_C4 = df_Beta.loc[df_Beta['electrode'] == 'C4']
Beta_Cz = df_Beta.loc[df_Beta['electrode'] == 'Cz']
Beta_F3 = df_Beta.loc[df_Beta['electrode'] == 'F3']
Beta_F4 = df_Beta.loc[df_Beta['electrode'] == 'F4']
Beta_F7 = df_Beta.loc[df_Beta['electrode'] == 'F7']
Beta_F8 = df_Beta.loc[df_Beta['electrode'] == 'F8']
Beta_FP1 = df_Beta.loc[df_Beta['electrode'] == 'FP1']
Beta_FP2 = df_Beta.loc[df_Beta['electrode'] == 'FP2']
Beta_Fz = df_Beta.loc[df_Beta['electrode'] == 'Fz']
Beta_O1 = df_Beta.loc[df_Beta['electrode'] == 'O1']
Beta_O2 = df_Beta.loc[df_Beta['electrode'] == 'O2']
Beta_P3 = df_Beta.loc[df_Beta['electrode'] == 'P3']
Beta_P4 = df_Beta.loc[df_Beta['electrode'] == 'P4']
Beta_Pz = df_Beta.loc[df_Beta['electrode'] == 'Pz']
Beta_T3 = df_Beta.loc[df_Beta['electrode'] == 'T3']
Beta_T4 = df_Beta.loc[df_Beta['electrode'] == 'T4']
Beta_T5 = df_Beta.loc[df_Beta['electrode'] == 'T5']
Beta_T6 = df_Beta.loc[df_Beta['electrode'] == 'T6']



In [9]:
df_Delta_women = df_Delta.loc[df_Delta['Gender'] == 1]
df_Delta_men = df_Delta.loc[df_Delta['Gender'] == 2]

df_Theta_women = df_Theta.loc[df_Theta['Gender'] == 1]
df_Theta_men = df_Theta.loc[df_Theta['Gender'] == 2]

df_Alpha_women = df_Alpha.loc[df_Alpha['Gender'] == 1]
df_Alpha_men = df_Alpha.loc[df_Alpha['Gender'] == 2]


df_Beta_women = df_Beta.loc[df_Beta['Gender'] == 1]
df_Beta_men = df_Beta.loc[df_Beta['Gender'] == 2]


In [10]:
#Make a df for each electrode for women in delta
Wdf_Delta_FP1  = df_Delta_women.loc[df_Delta_women['electrode'] == 'FP1']
Wdf_Delta_FP2  = df_Delta_women.loc[df_Delta_women['electrode'] == 'FP2']
Wdf_Delta_F3  = df_Delta_women.loc[df_Delta_women['electrode'] == 'F3']
Wdf_Delta_F4  = df_Delta_women.loc[df_Delta_women['electrode'] == 'F4']
Wdf_Delta_Fz  = df_Delta_women.loc[df_Delta_women['electrode'] == 'Fz']
Wdf_Delta_C3  = df_Delta_women.loc[df_Delta_women['electrode'] == 'C3']
Wdf_Delta_C4  = df_Delta_women.loc[df_Delta_women['electrode'] == 'C4']
Wdf_Delta_Cz  = df_Delta_women.loc[df_Delta_women['electrode'] == 'Cz']
Wdf_Delta_P3  = df_Delta_women.loc[df_Delta_women['electrode'] == 'P3']
Wdf_Delta_P4  = df_Delta_women.loc[df_Delta_women['electrode'] == 'P4']
Wdf_Delta_Pz  = df_Delta_women.loc[df_Delta_women['electrode'] == 'Pz']
Wdf_Delta_O1  = df_Delta_women.loc[df_Delta_women['electrode'] == 'O1']
Wdf_Delta_O2  = df_Delta_women.loc[df_Delta_women['electrode'] == 'O2']
Wdf_Delta_F7  = df_Delta_women.loc[df_Delta_women['electrode'] == 'F7']
Wdf_Delta_F8  = df_Delta_women.loc[df_Delta_women['electrode'] == 'F8']
Wdf_Delta_T3  = df_Delta_women.loc[df_Delta_women['electrode'] == 'T3']
Wdf_Delta_T4  = df_Delta_women.loc[df_Delta_women['electrode'] == 'T4']
Wdf_Delta_T5  = df_Delta_women.loc[df_Delta_women['electrode'] == 'T5']
Wdf_Delta_T6  = df_Delta_women.loc[df_Delta_women['electrode'] == 'T6']
DeltaW = pd.concat([Wdf_Delta_FP1, Wdf_Delta_FP2, Wdf_Delta_F3, Wdf_Delta_F4, Wdf_Delta_Fz, Wdf_Delta_C3,
                 Wdf_Delta_C4, Wdf_Delta_Cz, Wdf_Delta_P3, Wdf_Delta_P4, Wdf_Delta_Pz, Wdf_Delta_O1,
                 Wdf_Delta_O2, Wdf_Delta_F7, Wdf_Delta_F8, Wdf_Delta_T3,Wdf_Delta_T4,
                 Wdf_Delta_T5, Wdf_Delta_T6])

#Make a df for each electrode for men in delta
Mdf_Delta_FP1  = df_Delta_men.loc[df_Delta_men['electrode'] == 'FP1']
Mdf_Delta_FP2  = df_Delta_men.loc[df_Delta_men['electrode'] == 'FP2']
Mdf_Delta_F3  = df_Delta_men.loc[df_Delta_men['electrode'] == 'F3']
Mdf_Delta_F4  = df_Delta_men.loc[df_Delta_men['electrode'] == 'F4']
Mdf_Delta_Fz  = df_Delta_men.loc[df_Delta_men['electrode'] == 'Fz']
Mdf_Delta_C3  = df_Delta_men.loc[df_Delta_men['electrode'] == 'C3']
Mdf_Delta_C4  = df_Delta_men.loc[df_Delta_men['electrode'] == 'C4']
Mdf_Delta_Cz  = df_Delta_men.loc[df_Delta_men['electrode'] == 'Cz']
Mdf_Delta_P3  = df_Delta_men.loc[df_Delta_men['electrode'] == 'P3']
Mdf_Delta_P4  = df_Delta_men.loc[df_Delta_men['electrode'] == 'P4']
Mdf_Delta_Pz  = df_Delta_men.loc[df_Delta_men['electrode'] == 'Pz']
Mdf_Delta_O1  = df_Delta_men.loc[df_Delta_men['electrode'] == 'O1']
Mdf_Delta_O2  = df_Delta_men.loc[df_Delta_men['electrode'] == 'O2']
Mdf_Delta_F7  = df_Delta_men.loc[df_Delta_men['electrode'] == 'F7']
Mdf_Delta_F8  = df_Delta_men.loc[df_Delta_men['electrode'] == 'F8']
Mdf_Delta_T3  = df_Delta_men.loc[df_Delta_men['electrode'] == 'T3']
Mdf_Delta_T4  = df_Delta_men.loc[df_Delta_men['electrode'] == 'T4']
Mdf_Delta_T5  = df_Delta_men.loc[df_Delta_men['electrode'] == 'T5']
Mdf_Delta_T6  = df_Delta_men.loc[df_Delta_men['electrode'] == 'T6']

# now do the same for theta. 

#Make a df for each electrode for women in theta
Wdf_Theta_FP1  = df_Theta_women.loc[df_Theta_women['electrode'] == 'FP1']
Wdf_Theta_FP2  = df_Theta_women.loc[df_Theta_women['electrode'] == 'FP2']
Wdf_Theta_F3  = df_Theta_women.loc[df_Theta_women['electrode'] == 'F3']
Wdf_Theta_F4  = df_Theta_women.loc[df_Theta_women['electrode'] == 'F4']
Wdf_Theta_Fz  = df_Theta_women.loc[df_Theta_women['electrode'] == 'Fz']
Wdf_Theta_C3  = df_Theta_women.loc[df_Theta_women['electrode'] == 'C3']
Wdf_Theta_C4  = df_Theta_women.loc[df_Theta_women['electrode'] == 'C4']
Wdf_Theta_Cz  = df_Theta_women.loc[df_Theta_women['electrode'] == 'Cz']
Wdf_Theta_P3  = df_Theta_women.loc[df_Theta_women['electrode'] == 'P3']
Wdf_Theta_P4  = df_Theta_women.loc[df_Theta_women['electrode'] == 'P4']
Wdf_Theta_Pz  = df_Theta_women.loc[df_Theta_women['electrode'] == 'Pz']
Wdf_Theta_O1  = df_Theta_women.loc[df_Theta_women['electrode'] == 'O1']
Wdf_Theta_O2  = df_Theta_women.loc[df_Theta_women['electrode'] == 'O2']
Wdf_Theta_F7  = df_Theta_women.loc[df_Theta_women['electrode'] == 'F7']
Wdf_Theta_F8  = df_Theta_women.loc[df_Theta_women['electrode'] == 'F8']
Wdf_Theta_T3  = df_Theta_women.loc[df_Theta_women['electrode'] == 'T3']
Wdf_Theta_T4  = df_Theta_women.loc[df_Theta_women['electrode'] == 'T4']
Wdf_Theta_T5  = df_Theta_women.loc[df_Theta_women['electrode'] == 'T5']
Wdf_Theta_T6  = df_Theta_women.loc[df_Theta_women['electrode'] == 'T6']

#Make a df for each electrode for men in theta
Mdf_Theta_FP1  = df_Theta_men.loc[df_Theta_men['electrode'] == 'FP1']
Mdf_Theta_FP2  = df_Theta_men.loc[df_Theta_men['electrode'] == 'FP2']
Mdf_Theta_F3  = df_Theta_men.loc[df_Theta_men['electrode'] == 'F3']
Mdf_Theta_F4  = df_Theta_men.loc[df_Theta_men['electrode'] == 'F4']
Mdf_Theta_Fz  = df_Theta_men.loc[df_Theta_men['electrode'] == 'Fz']
Mdf_Theta_C3  = df_Theta_men.loc[df_Theta_men['electrode'] == 'C3']
Mdf_Theta_C4  = df_Theta_men.loc[df_Theta_men['electrode'] == 'C4']
Mdf_Theta_Cz  = df_Theta_men.loc[df_Theta_men['electrode'] == 'Cz']
Mdf_Theta_P3  = df_Theta_men.loc[df_Theta_men['electrode'] == 'P3']
Mdf_Theta_P4  = df_Theta_men.loc[df_Theta_men['electrode'] == 'P4']
Mdf_Theta_Pz  = df_Theta_men.loc[df_Theta_men['electrode'] == 'Pz']
Mdf_Theta_O1  = df_Theta_men.loc[df_Theta_men['electrode'] == 'O1']
Mdf_Theta_O2  = df_Theta_men.loc[df_Theta_men['electrode'] == 'O2']
Mdf_Theta_F7  = df_Theta_men.loc[df_Theta_men['electrode'] == 'F7']
Mdf_Theta_F8  = df_Theta_men.loc[df_Theta_men['electrode'] == 'F8']
Mdf_Theta_T3  = df_Theta_men.loc[df_Theta_men['electrode'] == 'T3']
Mdf_Theta_T4  = df_Theta_men.loc[df_Theta_men['electrode'] == 'T4']
Mdf_Theta_T5  = df_Theta_men.loc[df_Theta_men['electrode'] == 'T5']
Mdf_Theta_T6  = df_Theta_men.loc[df_Theta_men['electrode'] == 'T6']

# And do the same for Alpha

#Make a df for each electrode for women in alpha
Wdf_Alpha_FP1  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'FP1']
Wdf_Alpha_FP2  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'FP2']
Wdf_Alpha_F3  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'F3']
Wdf_Alpha_F4  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'F4']
Wdf_Alpha_Fz  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'Fz']
Wdf_Alpha_C3  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'C3']
Wdf_Alpha_C4  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'C4']
Wdf_Alpha_Cz  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'Cz']
Wdf_Alpha_P3  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'P3']
Wdf_Alpha_P4  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'P4']
Wdf_Alpha_Pz  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'Pz']
Wdf_Alpha_O1  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'O1']
Wdf_Alpha_O2  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'O2']
Wdf_Alpha_F7  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'F7']
Wdf_Alpha_F8  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'F8']
Wdf_Alpha_T3  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'T3']
Wdf_Alpha_T4  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'T4']
Wdf_Alpha_T5  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'T5']
Wdf_Alpha_T6  = df_Alpha_women.loc[df_Alpha_women['electrode'] == 'T6']

#Make a df for each electrode for men in alpha
Mdf_Alpha_FP1  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'FP1']
Mdf_Alpha_FP2  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'FP2']
Mdf_Alpha_F3  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'F3']
Mdf_Alpha_F4  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'F4']
Mdf_Alpha_Fz  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'Fz']
Mdf_Alpha_C3  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'C3']
Mdf_Alpha_C4  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'C4']
Mdf_Alpha_Cz  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'Cz']
Mdf_Alpha_P3  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'P3']
Mdf_Alpha_P4  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'P4']
Mdf_Alpha_Pz  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'Pz']
Mdf_Alpha_O1  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'O1']
Mdf_Alpha_O2  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'O2']
Mdf_Alpha_F7  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'F7']
Mdf_Alpha_F8  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'F8']
Mdf_Alpha_T3  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'T3']
Mdf_Alpha_T4  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'T4']
Mdf_Alpha_T5  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'T5']
Mdf_Alpha_T6  = df_Alpha_men.loc[df_Alpha_men['electrode'] == 'T6']

# And do the same for Beta

#Make a df for each electrode for women in beta
Wdf_Beta_FP1  = df_Beta_women.loc[df_Beta_women['electrode'] == 'FP1']
Wdf_Beta_FP2  = df_Beta_women.loc[df_Beta_women['electrode'] == 'FP2']
Wdf_Beta_F3  = df_Beta_women.loc[df_Beta_women['electrode'] == 'F3']
Wdf_Beta_F4  = df_Beta_women.loc[df_Beta_women['electrode'] == 'F4']
Wdf_Beta_Fz  = df_Beta_women.loc[df_Beta_women['electrode'] == 'Fz']
Wdf_Beta_C3  = df_Beta_women.loc[df_Beta_women['electrode'] == 'C3']
Wdf_Beta_C4  = df_Beta_women.loc[df_Beta_women['electrode'] == 'C4']
Wdf_Beta_Cz  = df_Beta_women.loc[df_Beta_women['electrode'] == 'Cz']
Wdf_Beta_P3  = df_Beta_women.loc[df_Beta_women['electrode'] == 'P3']
Wdf_Beta_P4  = df_Beta_women.loc[df_Beta_women['electrode'] == 'P4']
Wdf_Beta_Pz  = df_Beta_women.loc[df_Beta_women['electrode'] == 'Pz']
Wdf_Beta_O1  = df_Beta_women.loc[df_Beta_women['electrode'] == 'O1']
Wdf_Beta_O2  = df_Beta_women.loc[df_Beta_women['electrode'] == 'O2']
Wdf_Beta_F7  = df_Beta_women.loc[df_Beta_women['electrode'] == 'F7']
Wdf_Beta_F8  = df_Beta_women.loc[df_Beta_women['electrode'] == 'F8']
Wdf_Beta_T3  = df_Beta_women.loc[df_Beta_women['electrode'] == 'T3']
Wdf_Beta_T4  = df_Beta_women.loc[df_Beta_women['electrode'] == 'T4']
Wdf_Beta_T5  = df_Beta_women.loc[df_Beta_women['electrode'] == 'T5']
Wdf_Beta_T6  = df_Beta_women.loc[df_Beta_women['electrode'] == 'T6']

#Make a df for each electrode for men in beta
Mdf_Beta_FP1  = df_Beta_men.loc[df_Beta_men['electrode'] == 'FP1']
Mdf_Beta_FP2  = df_Beta_men.loc[df_Beta_men['electrode'] == 'FP2']
Mdf_Beta_F3  = df_Beta_men.loc[df_Beta_men['electrode'] == 'F3']
Mdf_Beta_F4  = df_Beta_men.loc[df_Beta_men['electrode'] == 'F4']
Mdf_Beta_Fz  = df_Beta_men.loc[df_Beta_men['electrode'] == 'Fz']
Mdf_Beta_C3  = df_Beta_men.loc[df_Beta_men['electrode'] == 'C3']
Mdf_Beta_C4  = df_Beta_men.loc[df_Beta_men['electrode'] == 'C4']
Mdf_Beta_Cz  = df_Beta_men.loc[df_Beta_men['electrode'] == 'Cz']
Mdf_Beta_P3  = df_Beta_men.loc[df_Beta_men['electrode'] == 'P3']
Mdf_Beta_P4  = df_Beta_men.loc[df_Beta_men['electrode'] == 'P4']
Mdf_Beta_Pz  = df_Beta_men.loc[df_Beta_men['electrode'] == 'Pz']
Mdf_Beta_O1  = df_Beta_men.loc[df_Beta_men['electrode'] == 'O1']
Mdf_Beta_O2  = df_Beta_men.loc[df_Beta_men['electrode'] == 'O2']
Mdf_Beta_F7  = df_Beta_men.loc[df_Beta_men['electrode'] == 'F7']
Mdf_Beta_F8  = df_Beta_men.loc[df_Beta_men['electrode'] == 'F8']
Mdf_Beta_T3  = df_Beta_men.loc[df_Beta_men['electrode'] == 'T3']
Mdf_Beta_T4  = df_Beta_men.loc[df_Beta_men['electrode'] == 'T4']
Mdf_Beta_T5  = df_Beta_men.loc[df_Beta_men['electrode'] == 'T5']
Mdf_Beta_T6  = df_Beta_men.loc[df_Beta_men['electrode'] == 'T6']



# COMPUTE P-VALUES FOR GENDER DIFFERENCES.

## Get DELTA p values

In [11]:
# prepare variable for pvals for future mask
pvalsD = []

Wdf_Delta_C3 = Wdf_Delta_C3[['fft_abs_power']]
Mdf_Delta_C3 = Mdf_Delta_C3[['fft_abs_power']]
data1 = Wdf_Delta_C3
data2 = Mdf_Delta_C3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values


Wdf_Delta_C4 = Wdf_Delta_C4[['fft_abs_power']]
Mdf_Delta_C4 = Mdf_Delta_C4[['fft_abs_power']]
data1 = Wdf_Delta_C4
data2 = Mdf_Delta_C4


# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values


Wdf_Delta_Cz = Wdf_Delta_Cz[['fft_abs_power']]
Mdf_Delta_Cz = Mdf_Delta_Cz[['fft_abs_power']]
data1 = Wdf_Delta_Cz
data2 = Mdf_Delta_Cz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values


Wdf_Delta_F3 = Wdf_Delta_F3[['fft_abs_power']]
Mdf_Delta_F3 = Mdf_Delta_F3[['fft_abs_power']]
data1 = Wdf_Delta_F3
data2 = Mdf_Delta_F3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values


Wdf_Delta_F4 = Wdf_Delta_F4[['fft_abs_power']]
Mdf_Delta_F4 = Mdf_Delta_F4[['fft_abs_power']]
data1 = Wdf_Delta_F4
data2 = Mdf_Delta_F4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values


Wdf_Delta_F7 = Wdf_Delta_F7[['fft_abs_power']]
Mdf_Delta_F7 = Mdf_Delta_F7[['fft_abs_power']]
data1 = Wdf_Delta_F7
data2 = Mdf_Delta_F7

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values


Wdf_Delta_F8 = Wdf_Delta_F8[['fft_abs_power']]
Mdf_Delta_F8 = Mdf_Delta_F8[['fft_abs_power']]
data1 = Wdf_Delta_F8
data2 = Mdf_Delta_F8

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values


Wdf_Delta_FP1 = Wdf_Delta_FP1[['fft_abs_power']]
Mdf_Delta_FP1 = Mdf_Delta_FP1[['fft_abs_power']]
data1 = Wdf_Delta_FP1
data2 = Mdf_Delta_FP1

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values


Wdf_Delta_FP2 = Wdf_Delta_FP2[['fft_abs_power']]
Mdf_Delta_FP2 = Mdf_Delta_FP2[['fft_abs_power']]
data1 = Wdf_Delta_FP2
data2 = Mdf_Delta_FP2

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values


Wdf_Delta_Fz = Wdf_Delta_Fz[['fft_abs_power']]
Mdf_Delta_Fz = Mdf_Delta_Fz[['fft_abs_power']]
data1 = Wdf_Delta_Fz
data2 = Mdf_Delta_Fz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values


Wdf_Delta_O1 = Wdf_Delta_O1[['fft_abs_power']]
Mdf_Delta_O1 = Mdf_Delta_O1[['fft_abs_power']]
data1 = Wdf_Delta_O1
data2 = Mdf_Delta_O1

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values


Wdf_Delta_O2 = Wdf_Delta_O2[['fft_abs_power']]
Mdf_Delta_O2 = Mdf_Delta_O2[['fft_abs_power']]
data1 = Wdf_Delta_O2
data2 = Mdf_Delta_O2

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values


Wdf_Delta_P3 = Wdf_Delta_P3[['fft_abs_power']]
Mdf_Delta_P3 = Mdf_Delta_P3[['fft_abs_power']]
data1 = Wdf_Delta_P3
data2 = Mdf_Delta_P3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values


Wdf_Delta_P4 = Wdf_Delta_P4[['fft_abs_power']]
Mdf_Delta_P4 = Mdf_Delta_P4[['fft_abs_power']]
data1 = Wdf_Delta_P4
data2 = Mdf_Delta_P4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values


Wdf_Delta_Pz = Wdf_Delta_Pz[['fft_abs_power']]
Mdf_Delta_Pz = Mdf_Delta_Pz[['fft_abs_power']]
data1 = Wdf_Delta_Pz
data2 = Mdf_Delta_Pz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values


Wdf_Delta_T3 = Wdf_Delta_T3[['fft_abs_power']]
Mdf_Delta_T3 = Mdf_Delta_T3[['fft_abs_power']]
data1 = Wdf_Delta_T3
data2 = Mdf_Delta_T3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values


Wdf_Delta_T4 = Wdf_Delta_T4[['fft_abs_power']]
Mdf_Delta_T4 = Mdf_Delta_T4[['fft_abs_power']]
data1 = Wdf_Delta_T4
data2 = Mdf_Delta_T4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values


Wdf_Delta_T5 = Wdf_Delta_T5[['fft_abs_power']]
Mdf_Delta_T5 = Mdf_Delta_T5[['fft_abs_power']]
data1 = Wdf_Delta_T5
data2 = Mdf_Delta_T5

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values


Wdf_Delta_T6 = Wdf_Delta_T6[['fft_abs_power']]
Mdf_Delta_T6 = Mdf_Delta_T6[['fft_abs_power']]
data1 = Wdf_Delta_T6
data2 = Mdf_Delta_T6

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsD.append(p) #truc pour updater l'array des p-values
print(pvalsD)

Statistics=873.000, p=0.038
Statistics=893.000, p=0.052
Statistics=780.000, p=0.007
Statistics=956.000, p=0.124
Statistics=878.000, p=0.041
Statistics=1006.000, p=0.217
Statistics=1071.000, p=0.383
Statistics=833.000, p=0.019
Statistics=802.000, p=0.011
Statistics=882.000, p=0.044
Statistics=835.000, p=0.020
Statistics=724.000, p=0.002
Statistics=930.000, p=0.088
Statistics=853.000, p=0.027
Statistics=816.000, p=0.014
Statistics=990.000, p=0.183
Statistics=1001.000, p=0.206
Statistics=878.000, p=0.041
Statistics=849.000, p=0.025
[0.03791061496843451, 0.05194656622500621, 0.006770049360250207, 0.12378130423729783, 0.04109089156413973, 0.2167274950594471, 0.3826999415565635, 0.019046071970996305, 0.010579932017209262, 0.043788023714835284, 0.019749898906718227, 0.001944703788623363, 0.08846819474791551, 0.027135756943425987, 0.013879316987251004, 0.18335528411542018, 0.20593912100451572, 0.04109089156413973, 0.025320886995478343]


In [12]:
from statsmodels.sandbox.stats.multicomp import multipletests
pvals = pvalsD

# Create a list of the adjusted p-values
p_adjusted_D = multipletests(pvals, alpha=0.05, method='bonferroni')

# Print the resulting conclusions
print(p_adjusted_D)
p_adjusted_D = p_adjusted_D[1]
p_adjusted_D


(array([False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False]), array([0.72030168, 0.98698476, 0.12863094, 1.        , 0.78072694,
       1.        , 1.        , 0.36187537, 0.20101871, 0.83197245,
       0.37524808, 0.03694937, 1.        , 0.51557938, 0.26370702,
       1.        , 1.        , 0.78072694, 0.48109685]), 0.0026960063028712566, 0.002631578947368421)


array([0.72030168, 0.98698476, 0.12863094, 1.        , 0.78072694,
       1.        , 1.        , 0.36187537, 0.20101871, 0.83197245,
       0.37524808, 0.03694937, 1.        , 0.51557938, 0.26370702,
       1.        , 1.        , 0.78072694, 0.48109685])

## Get THETA p-values

In [13]:
pvalsT = []
Wdf_Theta_C3 = Wdf_Theta_C3[['fft_abs_power']]
Mdf_Theta_C3 = Mdf_Theta_C3[['fft_abs_power']]
data1 = Wdf_Theta_C3
data2 = Mdf_Theta_C3
# important, j'ai mis ça pour avoir un array qui s'update à chaque test, qui va 
#inclure toutes les p-values pour le masque des topo. 

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values


Wdf_Theta_C4 = Wdf_Theta_C4[['fft_abs_power']]
Mdf_Theta_C4 = Mdf_Theta_C4[['fft_abs_power']]
data1 = Wdf_Theta_C4
data2 = Mdf_Theta_C4


# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values


Wdf_Theta_Cz = Wdf_Theta_Cz[['fft_abs_power']]
Mdf_Theta_Cz = Mdf_Theta_Cz[['fft_abs_power']]
data1 = Wdf_Theta_Cz
data2 = Mdf_Theta_Cz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values

Wdf_Theta_F3 = Wdf_Theta_F3[['fft_abs_power']]
Mdf_Theta_F3 = Mdf_Theta_F3[['fft_abs_power']]
data1 = Wdf_Theta_F3
data2 = Mdf_Theta_F3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values


Wdf_Theta_F4 = Wdf_Theta_F4[['fft_abs_power']]
Mdf_Theta_F4 = Mdf_Theta_F4[['fft_abs_power']]
data1 = Wdf_Theta_F4
data2 = Mdf_Theta_F4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values


Wdf_Theta_F7 = Wdf_Theta_F7[['fft_abs_power']]
Mdf_Theta_F7 = Mdf_Theta_F7[['fft_abs_power']]
data1 = Wdf_Theta_F7
data2 = Mdf_Theta_F7

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values


Wdf_Theta_F8 = Wdf_Theta_F8[['fft_abs_power']]
Mdf_Theta_F8 = Mdf_Theta_F8[['fft_abs_power']]
data1 = Wdf_Theta_F8
data2 = Mdf_Theta_F8

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values

Wdf_Theta_FP1 = Wdf_Theta_FP1[['fft_abs_power']]
Mdf_Theta_FP1 = Mdf_Theta_FP1[['fft_abs_power']]
data1 = Wdf_Theta_FP1
data2 = Mdf_Theta_FP1

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values


Wdf_Theta_FP2 = Wdf_Theta_FP2[['fft_abs_power']]
Mdf_Theta_FP2 = Mdf_Theta_FP2[['fft_abs_power']]
data1 = Wdf_Theta_FP2
data2 = Mdf_Theta_FP2

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values


Wdf_Theta_Fz = Wdf_Theta_Fz[['fft_abs_power']]
Mdf_Theta_Fz = Mdf_Theta_Fz[['fft_abs_power']]
data1 = Wdf_Theta_Fz
data2 = Mdf_Theta_Fz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values


Wdf_Theta_O1 = Wdf_Theta_O1[['fft_abs_power']]
Mdf_Theta_O1 = Mdf_Theta_O1[['fft_abs_power']]
data1 = Wdf_Theta_O1
data2 = Mdf_Theta_O1

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values


Wdf_Theta_O2 = Wdf_Theta_O2[['fft_abs_power']]
Mdf_Theta_O2 = Mdf_Theta_O2[['fft_abs_power']]
data1 = Wdf_Theta_O2
data2 = Mdf_Theta_O2

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values


Wdf_Theta_P3 = Wdf_Theta_P3[['fft_abs_power']]
Mdf_Theta_P3 = Mdf_Theta_P3[['fft_abs_power']]
data1 = Wdf_Theta_P3
data2 = Mdf_Theta_P3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values


Wdf_Theta_P4 = Wdf_Theta_P4[['fft_abs_power']]
Mdf_Theta_P4 = Mdf_Theta_P4[['fft_abs_power']]
data1 = Wdf_Theta_P4
data2 = Mdf_Theta_P4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values


Wdf_Theta_Pz = Wdf_Theta_Pz[['fft_abs_power']]
Mdf_Theta_Pz = Mdf_Theta_Pz[['fft_abs_power']]
data1 = Wdf_Theta_Pz
data2 = Mdf_Theta_Pz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values

Wdf_Theta_T3 = Wdf_Theta_T3[['fft_abs_power']]
Mdf_Theta_T3 = Mdf_Theta_T3[['fft_abs_power']]
data1 = Wdf_Theta_T3
data2 = Mdf_Theta_T3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values

Wdf_Theta_T4 = Wdf_Theta_T4[['fft_abs_power']]
Mdf_Theta_T4 = Mdf_Theta_T4[['fft_abs_power']]
data1 = Wdf_Theta_T4
data2 = Mdf_Theta_T4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values

Wdf_Theta_T5 = Wdf_Theta_T5[['fft_abs_power']]
Mdf_Theta_T5 = Mdf_Theta_T5[['fft_abs_power']]
data1 = Wdf_Theta_T5
data2 = Mdf_Theta_T5

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values


Wdf_Theta_T6 = Wdf_Theta_T6[['fft_abs_power']]
Mdf_Theta_T6 = Mdf_Theta_T6[['fft_abs_power']]
data1 = Wdf_Theta_T6
data2 = Mdf_Theta_T6

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsT.append(p) #truc pour updater l'array des p-values
print(pvalsT)


Statistics=958.000, p=0.127
Statistics=961.000, p=0.132
Statistics=843.000, p=0.023
Statistics=1015.000, p=0.237
Statistics=989.000, p=0.181
Statistics=1072.000, p=0.386
Statistics=1104.000, p=0.479
Statistics=1096.000, p=0.455
Statistics=1079.000, p=0.406
Statistics=988.000, p=0.179
Statistics=900.000, p=0.058
Statistics=854.000, p=0.028
Statistics=966.000, p=0.140
Statistics=959.000, p=0.128
Statistics=912.000, p=0.069
Statistics=1003.000, p=0.210
Statistics=1031.000, p=0.275
Statistics=956.000, p=0.124
Statistics=953.000, p=0.119
[0.12685799058132313, 0.13157215963865548, 0.02278998002863893, 0.23694821531284427, 0.1813817152938027, 0.3855495784002121, 0.479176880663588, 0.4554517367685357, 0.4056621641571505, 0.1794215003281585, 0.05773838257004888, 0.02760606010035468, 0.13969503168413933, 0.12841613615369873, 0.06883458277179937, 0.21021572665402882, 0.2753222037172606, 0.12378130423729783, 0.11926486694391075]


In [14]:
pvals = pvalsT

# Create a list of the adjusted p-values
p_adjusted_T = multipletests(pvals, alpha=0.05, method='bonferroni')

# Print the resulting conclusions
print(p_adjusted_T)
p_adjusted_T = p_adjusted_T[1]
p_adjusted_T

(array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False]), array([1.        , 1.        , 0.43300962, 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.52451514, 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        ]), 0.0026960063028712566, 0.002631578947368421)


array([1.        , 1.        , 0.43300962, 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.52451514, 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        ])

## Get ALPHA p-values

In [15]:
pvalsA = []
Wdf_Alpha_C3 = Wdf_Alpha_C3[['fft_abs_power']]
Mdf_Alpha_C3 = Mdf_Alpha_C3[['fft_abs_power']]
data1 = Wdf_Alpha_C3
data2 = Mdf_Alpha_C3
# important, j'ai mis ça pour avoir un array qui s'update à chaque test, qui va 
#inclure toutes les p-values pour le masque des topo. 

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values


Wdf_Alpha_C4 = Wdf_Alpha_C4[['fft_abs_power']]
Mdf_Alpha_C4 = Mdf_Alpha_C4[['fft_abs_power']]
data1 = Wdf_Alpha_C4
data2 = Mdf_Alpha_C4


# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values


Wdf_Alpha_Cz = Wdf_Alpha_Cz[['fft_abs_power']]
Mdf_Alpha_Cz = Mdf_Alpha_Cz[['fft_abs_power']]
data1 = Wdf_Alpha_Cz
data2 = Mdf_Alpha_Cz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values

Wdf_Alpha_F3 = Wdf_Alpha_F3[['fft_abs_power']]
Mdf_Alpha_F3 = Mdf_Alpha_F3[['fft_abs_power']]
data1 = Wdf_Alpha_F3
data2 = Mdf_Alpha_F3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values


Wdf_Alpha_F4 = Wdf_Alpha_F4[['fft_abs_power']]
Mdf_Alpha_F4 = Mdf_Alpha_F4[['fft_abs_power']]
data1 = Wdf_Alpha_F4
data2 = Mdf_Alpha_F4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values


Wdf_Alpha_F7 = Wdf_Alpha_F7[['fft_abs_power']]
Mdf_Alpha_F7 = Mdf_Alpha_F7[['fft_abs_power']]
data1 = Wdf_Alpha_F7
data2 = Mdf_Alpha_F7

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values


Wdf_Alpha_F8 = Wdf_Alpha_F8[['fft_abs_power']]
Mdf_Alpha_F8 = Mdf_Alpha_F8[['fft_abs_power']]
data1 = Wdf_Alpha_F8
data2 = Mdf_Alpha_F8

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values

Wdf_Alpha_FP1 = Wdf_Alpha_FP1[['fft_abs_power']]
Mdf_Alpha_FP1 = Mdf_Alpha_FP1[['fft_abs_power']]
data1 = Wdf_Alpha_FP1
data2 = Mdf_Alpha_FP1

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values


Wdf_Alpha_FP2 = Wdf_Alpha_FP2[['fft_abs_power']]
Mdf_Alpha_FP2 = Mdf_Alpha_FP2[['fft_abs_power']]
data1 = Wdf_Alpha_FP2
data2 = Mdf_Alpha_FP2

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values


Wdf_Alpha_Fz = Wdf_Alpha_Fz[['fft_abs_power']]
Mdf_Alpha_Fz = Mdf_Alpha_Fz[['fft_abs_power']]
data1 = Wdf_Alpha_Fz
data2 = Mdf_Alpha_Fz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values


Wdf_Alpha_O1 = Wdf_Alpha_O1[['fft_abs_power']]
Mdf_Alpha_O1 = Mdf_Alpha_O1[['fft_abs_power']]
data1 = Wdf_Alpha_O1
data2 = Mdf_Alpha_O1

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values


Wdf_Alpha_O2 = Wdf_Alpha_O2[['fft_abs_power']]
Mdf_Alpha_O2 = Mdf_Alpha_O2[['fft_abs_power']]
data1 = Wdf_Alpha_O2
data2 = Mdf_Alpha_O2

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values


Wdf_Alpha_P3 = Wdf_Alpha_P3[['fft_abs_power']]
Mdf_Alpha_P3 = Mdf_Alpha_P3[['fft_abs_power']]
data1 = Wdf_Alpha_P3
data2 = Mdf_Alpha_P3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values


Wdf_Alpha_P4 = Wdf_Alpha_P4[['fft_abs_power']]
Mdf_Alpha_P4 = Mdf_Alpha_P4[['fft_abs_power']]
data1 = Wdf_Alpha_P4
data2 = Mdf_Alpha_P4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values


Wdf_Alpha_Pz = Wdf_Alpha_Pz[['fft_abs_power']]
Mdf_Alpha_Pz = Mdf_Alpha_Pz[['fft_abs_power']]
data1 = Wdf_Alpha_Pz
data2 = Mdf_Alpha_Pz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values

Wdf_Alpha_T3 = Wdf_Alpha_T3[['fft_abs_power']]
Mdf_Alpha_T3 = Mdf_Alpha_T3[['fft_abs_power']]
data1 = Wdf_Alpha_T3
data2 = Mdf_Alpha_T3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values

Wdf_Alpha_T4 = Wdf_Alpha_T4[['fft_abs_power']]
Mdf_Alpha_T4 = Mdf_Alpha_T4[['fft_abs_power']]
data1 = Wdf_Alpha_T4
data2 = Mdf_Alpha_T4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values

Wdf_Alpha_T5 = Wdf_Alpha_T5[['fft_abs_power']]
Mdf_Alpha_T5 = Mdf_Alpha_T5[['fft_abs_power']]
data1 = Wdf_Alpha_T5
data2 = Mdf_Alpha_T5

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values


Wdf_Alpha_T6 = Wdf_Alpha_T6[['fft_abs_power']]
Mdf_Alpha_T6 = Mdf_Alpha_T6[['fft_abs_power']]
data1 = Wdf_Alpha_T6
data2 = Mdf_Alpha_T6

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsA.append(p) #truc pour updater l'array des p-values
print(pvalsA)


Statistics=1078.000, p=0.403
Statistics=1053.000, p=0.333
Statistics=1058.000, p=0.346
Statistics=1092.000, p=0.444
Statistics=1095.000, p=0.452
Statistics=1019.000, p=0.246
Statistics=993.000, p=0.189
Statistics=1063.000, p=0.360
Statistics=1070.000, p=0.380
Statistics=1109.000, p=0.494
Statistics=1098.000, p=0.461
Statistics=1076.000, p=0.397
Statistics=1049.000, p=0.322
Statistics=1039.000, p=0.296
Statistics=1092.000, p=0.444
Statistics=1105.000, p=0.482
Statistics=1044.000, p=0.309
Statistics=1066.000, p=0.369
Statistics=1066.000, p=0.369
[0.40277228277963995, 0.33262500718794863, 0.346282510802157, 0.4436431794279856, 0.4524954890388321, 0.24625756432678375, 0.18935590957554244, 0.3601428960901424, 0.3798566409914539, 0.49404805411771585, 0.4613714758260911, 0.3970085773874021, 0.321855683507642, 0.2955939876071212, 0.4436431794279856, 0.48214946045271767, 0.30860245435788114, 0.3685496613777003, 0.3685496613777003]


In [16]:
pvals = pvalsA

# Create a list of the adjusted p-values
p_adjusted_A = multipletests(pvals, alpha=0.05, method='bonferroni')

# Print the resulting conclusions
print(p_adjusted_A)
p_adjusted_A = p_adjusted_A[1]
p_adjusted_A

(array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False]), array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1.]), 0.0026960063028712566, 0.002631578947368421)


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1.])

## Get BETA p-values

In [17]:
pvalsB = []
Wdf_Beta_C3 = Wdf_Beta_C3[['fft_abs_power']]
Mdf_Beta_C3 = Mdf_Beta_C3[['fft_abs_power']]
data1 = Wdf_Beta_C3
data2 = Mdf_Beta_C3
# important, j'ai mis ça pour avoir un array qui s'update à chaque test, qui va 
#inclure toutes les p-values pour le masque des topo. 

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values


Wdf_Beta_C4 = Wdf_Beta_C4[['fft_abs_power']]
Mdf_Beta_C4 = Mdf_Beta_C4[['fft_abs_power']]
data1 = Wdf_Beta_C4
data2 = Mdf_Beta_C4


# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values


Wdf_Beta_Cz = Wdf_Beta_Cz[['fft_abs_power']]
Mdf_Beta_Cz = Mdf_Beta_Cz[['fft_abs_power']]
data1 = Wdf_Beta_Cz
data2 = Mdf_Beta_Cz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values

Wdf_Beta_F3 = Wdf_Beta_F3[['fft_abs_power']]
Mdf_Beta_F3 = Mdf_Beta_F3[['fft_abs_power']]
data1 = Wdf_Beta_F3
data2 = Mdf_Beta_F3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values


Wdf_Beta_F4 = Wdf_Beta_F4[['fft_abs_power']]
Mdf_Beta_F4 = Mdf_Beta_F4[['fft_abs_power']]
data1 = Wdf_Beta_F4
data2 = Mdf_Beta_F4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values


Wdf_Beta_F7 = Wdf_Beta_F7[['fft_abs_power']]
Mdf_Beta_F7 = Mdf_Beta_F7[['fft_abs_power']]
data1 = Wdf_Beta_F7
data2 = Mdf_Beta_F7

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values


Wdf_Beta_F8 = Wdf_Beta_F8[['fft_abs_power']]
Mdf_Beta_F8 = Mdf_Beta_F8[['fft_abs_power']]
data1 = Wdf_Beta_F8
data2 = Mdf_Beta_F8

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values

Wdf_Beta_FP1 = Wdf_Beta_FP1[['fft_abs_power']]
Mdf_Beta_FP1 = Mdf_Beta_FP1[['fft_abs_power']]
data1 = Wdf_Beta_FP1
data2 = Mdf_Beta_FP1

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values


Wdf_Beta_FP2 = Wdf_Beta_FP2[['fft_abs_power']]
Mdf_Beta_FP2 = Mdf_Beta_FP2[['fft_abs_power']]
data1 = Wdf_Beta_FP2
data2 = Mdf_Beta_FP2

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values


Wdf_Beta_Fz = Wdf_Beta_Fz[['fft_abs_power']]
Mdf_Beta_Fz = Mdf_Beta_Fz[['fft_abs_power']]
data1 = Wdf_Beta_Fz
data2 = Mdf_Beta_Fz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values


Wdf_Beta_O1 = Wdf_Beta_O1[['fft_abs_power']]
Mdf_Beta_O1 = Mdf_Beta_O1[['fft_abs_power']]
data1 = Wdf_Beta_O1
data2 = Mdf_Beta_O1

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values


Wdf_Beta_O2 = Wdf_Beta_O2[['fft_abs_power']]
Mdf_Beta_O2 = Mdf_Beta_O2[['fft_abs_power']]
data1 = Wdf_Beta_O2
data2 = Mdf_Beta_O2

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values


Wdf_Beta_P3 = Wdf_Beta_P3[['fft_abs_power']]
Mdf_Beta_P3 = Mdf_Beta_P3[['fft_abs_power']]
data1 = Wdf_Beta_P3
data2 = Mdf_Beta_P3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values


Wdf_Beta_P4 = Wdf_Beta_P4[['fft_abs_power']]
Mdf_Beta_P4 = Mdf_Beta_P4[['fft_abs_power']]
data1 = Wdf_Beta_P4
data2 = Mdf_Beta_P4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values


Wdf_Beta_Pz = Wdf_Beta_Pz[['fft_abs_power']]
Mdf_Beta_Pz = Mdf_Beta_Pz[['fft_abs_power']]
data1 = Wdf_Beta_Pz
data2 = Mdf_Beta_Pz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values

Wdf_Beta_T3 = Wdf_Beta_T3[['fft_abs_power']]
Mdf_Beta_T3 = Mdf_Beta_T3[['fft_abs_power']]
data1 = Wdf_Beta_T3
data2 = Mdf_Beta_T3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values

Wdf_Beta_T4 = Wdf_Beta_T4[['fft_abs_power']]
Mdf_Beta_T4 = Mdf_Beta_T4[['fft_abs_power']]
data1 = Wdf_Beta_T4
data2 = Mdf_Beta_T4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values

Wdf_Beta_T5 = Wdf_Beta_T5[['fft_abs_power']]
Mdf_Beta_T5 = Mdf_Beta_T5[['fft_abs_power']]
data1 = Wdf_Beta_T5
data2 = Mdf_Beta_T5

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values


Wdf_Beta_T6 = Wdf_Beta_T6[['fft_abs_power']]
Mdf_Beta_T6 = Mdf_Beta_T6[['fft_abs_power']]
data1 = Wdf_Beta_T6
data2 = Mdf_Beta_T6

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
pvalsB.append(p) #truc pour updater l'array des p-values
print(pvalsB)


Statistics=909.000, p=0.066
Statistics=853.000, p=0.027
Statistics=861.000, p=0.031
Statistics=859.000, p=0.030
Statistics=766.000, p=0.005
Statistics=865.000, p=0.033
Statistics=945.000, p=0.108
Statistics=750.000, p=0.004
Statistics=648.000, p=0.000
Statistics=928.000, p=0.086
Statistics=874.000, p=0.039
Statistics=848.000, p=0.025
Statistics=857.000, p=0.029
Statistics=853.000, p=0.027
Statistics=869.000, p=0.036
Statistics=697.000, p=0.001
Statistics=793.000, p=0.009
Statistics=776.000, p=0.006
Statistics=749.000, p=0.003
[0.06591727351461892, 0.027135756943425987, 0.031091846670344866, 0.03006061279365695, 0.005031201424229325, 0.033242183150160885, 0.10779331218631855, 0.0035402267273711525, 0.0002762201452030413, 0.0861001195562352, 0.03853007415621581, 0.02488340654331364, 0.029057900493094153, 0.027135756943425987, 0.03551354704222218, 0.0010061431655395432, 0.008839840910079881, 0.006225843008995167, 0.003461804807451839]


In [18]:
pvals = pvalsB

# Create a list of the adjusted p-values
p_adjusted_B = multipletests(pvals, alpha=0.05, method='bonferroni')

# Print the resulting conclusions
print(p_adjusted_B)
p_adjusted_B = p_adjusted_B[1]
p_adjusted_B

(array([False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False,  True, False, False,
       False]), array([1.        , 0.51557938, 0.59074509, 0.57115164, 0.09559283,
       0.63160148, 1.        , 0.06726431, 0.00524818, 1.        ,
       0.73207141, 0.47278472, 0.55210011, 0.51557938, 0.67475739,
       0.01911672, 0.16795698, 0.11829102, 0.06577429]), 0.0026960063028712566, 0.002631578947368421)


array([1.        , 0.51557938, 0.59074509, 0.57115164, 0.09559283,
       0.63160148, 1.        , 0.06726431, 0.00524818, 1.        ,
       0.73207141, 0.47278472, 0.55210011, 0.51557938, 0.67475739,
       0.01911672, 0.16795698, 0.11829102, 0.06577429])

In [32]:
pvals_adjusted_D = pd.DataFrame(p_adjusted_D) 
pvals_adjusted_T = pd.DataFrame(p_adjusted_T) 
pvals_adjusted_A = pd.DataFrame(p_adjusted_A) 
pvals_adjusted_B = pd.DataFrame(p_adjusted_B) 
pvalsss = pd.concat([pvals_adjusted_D, pvals_adjusted_T, pvals_adjusted_A, pvals_adjusted_B])
#gender_pvals = np.asarray(pvalsss)
pvalsss.to_csv('Data/Gender_pvals')
gender_pvals = np.asarray(pvalsss)

# COMPUTING P-VALUES FOR DIFFERENCES IN ADHD SUBTYPE

In [21]:
df_Delta_inat = df_Delta.loc[df_Delta['adhdtype'] == 1]
df_Delta_mixed = df_Delta.loc[df_Delta['adhdtype'] == 2]

df_Theta_inat = df_Theta.loc[df_Theta['adhdtype'] == 1]
df_Theta_mixed = df_Theta.loc[df_Theta['adhdtype'] == 2]

df_Alpha_inat = df_Alpha.loc[df_Alpha['adhdtype'] == 1]
df_Alpha_mixed = df_Alpha.loc[df_Alpha['adhdtype'] == 2]


df_Beta_inat = df_Beta.loc[df_Beta['adhdtype'] == 1]
df_Beta_mixed = df_Beta.loc[df_Beta['adhdtype'] == 2]

In [22]:
#Make a df for each electrode for inat in delta
Inatdf_Delta_FP1  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'FP1']
Inatdf_Delta_FP2  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'FP2']
Inatdf_Delta_F3  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'F3']
Inatdf_Delta_F4  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'F4']
Inatdf_Delta_Fz  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'Fz']
Inatdf_Delta_C3  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'C3']
Inatdf_Delta_C4  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'C4']
Inatdf_Delta_Cz  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'Cz']
Inatdf_Delta_P3  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'P3']
Inatdf_Delta_P4  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'P4']
Inatdf_Delta_Pz  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'Pz']
Inatdf_Delta_O1  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'O1']
Inatdf_Delta_O2  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'O2']
Inatdf_Delta_F7  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'F7']
Inatdf_Delta_F8  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'F8']
Inatdf_Delta_T3  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'T3']
Inatdf_Delta_T4  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'T4']
Inatdf_Delta_T5  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'T5']
Inatdf_Delta_T6  = df_Delta_inat.loc[df_Delta_inat['electrode'] == 'T6']
DeltaW = pd.concat([Inatdf_Delta_FP1, Inatdf_Delta_FP2, Inatdf_Delta_F3, Inatdf_Delta_F4, Inatdf_Delta_Fz, Inatdf_Delta_C3,
                 Inatdf_Delta_C4, Inatdf_Delta_Cz, Inatdf_Delta_P3, Inatdf_Delta_P4, Inatdf_Delta_Pz, Inatdf_Delta_O1,
                 Inatdf_Delta_O2, Inatdf_Delta_F7, Inatdf_Delta_F8, Inatdf_Delta_T3,Inatdf_Delta_T4,
                 Inatdf_Delta_T5, Inatdf_Delta_T6])

#Make a df for each electrode for mixed in delta
Mixdf_Delta_FP1  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'FP1']
Mixdf_Delta_FP2  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'FP2']
Mixdf_Delta_F3  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'F3']
Mixdf_Delta_F4  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'F4']
Mixdf_Delta_Fz  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'Fz']
Mixdf_Delta_C3  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'C3']
Mixdf_Delta_C4  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'C4']
Mixdf_Delta_Cz  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'Cz']
Mixdf_Delta_P3  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'P3']
Mixdf_Delta_P4  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'P4']
Mixdf_Delta_Pz  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'Pz']
Mixdf_Delta_O1  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'O1']
Mixdf_Delta_O2  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'O2']
Mixdf_Delta_F7  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'F7']
Mixdf_Delta_F8  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'F8']
Mixdf_Delta_T3  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'T3']
Mixdf_Delta_T4  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'T4']
Mixdf_Delta_T5  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'T5']
Mixdf_Delta_T6  = df_Delta_mixed.loc[df_Delta_mixed['electrode'] == 'T6']

# now do the same for theta. 

#Make a df for each electrode for inat in theta
Inatdf_Theta_FP1  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'FP1']
Inatdf_Theta_FP2  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'FP2']
Inatdf_Theta_F3  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'F3']
Inatdf_Theta_F4  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'F4']
Inatdf_Theta_Fz  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'Fz']
Inatdf_Theta_C3  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'C3']
Inatdf_Theta_C4  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'C4']
Inatdf_Theta_Cz  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'Cz']
Inatdf_Theta_P3  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'P3']
Inatdf_Theta_P4  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'P4']
Inatdf_Theta_Pz  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'Pz']
Inatdf_Theta_O1  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'O1']
Inatdf_Theta_O2  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'O2']
Inatdf_Theta_F7  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'F7']
Inatdf_Theta_F8  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'F8']
Inatdf_Theta_T3  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'T3']
Inatdf_Theta_T4  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'T4']
Inatdf_Theta_T5  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'T5']
Inatdf_Theta_T6  = df_Theta_inat.loc[df_Theta_inat['electrode'] == 'T6']

#Make a df for each electrode for mixed in theta
Mixdf_Theta_FP1  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'FP1']
Mixdf_Theta_FP2  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'FP2']
Mixdf_Theta_F3  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'F3']
Mixdf_Theta_F4  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'F4']
Mixdf_Theta_Fz  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'Fz']
Mixdf_Theta_C3  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'C3']
Mixdf_Theta_C4  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'C4']
Mixdf_Theta_Cz  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'Cz']
Mixdf_Theta_P3  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'P3']
Mixdf_Theta_P4  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'P4']
Mixdf_Theta_Pz  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'Pz']
Mixdf_Theta_O1  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'O1']
Mixdf_Theta_O2  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'O2']
Mixdf_Theta_F7  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'F7']
Mixdf_Theta_F8  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'F8']
Mixdf_Theta_T3  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'T3']
Mixdf_Theta_T4  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'T4']
Mixdf_Theta_T5  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'T5']
Mixdf_Theta_T6  = df_Theta_mixed.loc[df_Theta_mixed['electrode'] == 'T6']

# And do the same for Alpha

#Make a df for each electrode for inat in alpha
Inatdf_Alpha_FP1  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'FP1']
Inatdf_Alpha_FP2  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'FP2']
Inatdf_Alpha_F3  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'F3']
Inatdf_Alpha_F4  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'F4']
Inatdf_Alpha_Fz  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'Fz']
Inatdf_Alpha_C3  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'C3']
Inatdf_Alpha_C4  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'C4']
Inatdf_Alpha_Cz  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'Cz']
Inatdf_Alpha_P3  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'P3']
Inatdf_Alpha_P4  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'P4']
Inatdf_Alpha_Pz  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'Pz']
Inatdf_Alpha_O1  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'O1']
Inatdf_Alpha_O2  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'O2']
Inatdf_Alpha_F7  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'F7']
Inatdf_Alpha_F8  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'F8']
Inatdf_Alpha_T3  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'T3']
Inatdf_Alpha_T4  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'T4']
Inatdf_Alpha_T5  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'T5']
Inatdf_Alpha_T6  = df_Alpha_inat.loc[df_Alpha_inat['electrode'] == 'T6']

#Make a df for each electrode for mixed in alpha
Mixdf_Alpha_FP1  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'FP1']
Mixdf_Alpha_FP2  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'FP2']
Mixdf_Alpha_F3  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'F3']
Mixdf_Alpha_F4  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'F4']
Mixdf_Alpha_Fz  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'Fz']
Mixdf_Alpha_C3  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'C3']
Mixdf_Alpha_C4  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'C4']
Mixdf_Alpha_Cz  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'Cz']
Mixdf_Alpha_P3  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'P3']
Mixdf_Alpha_P4  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'P4']
Mixdf_Alpha_Pz  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'Pz']
Mixdf_Alpha_O1  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'O1']
Mixdf_Alpha_O2  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'O2']
Mixdf_Alpha_F7  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'F7']
Mixdf_Alpha_F8  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'F8']
Mixdf_Alpha_T3  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'T3']
Mixdf_Alpha_T4  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'T4']
Mixdf_Alpha_T5  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'T5']
Mixdf_Alpha_T6  = df_Alpha_mixed.loc[df_Alpha_mixed['electrode'] == 'T6']

# And do the same for Beta

#Make a df for each electrode for inat in beta
Inatdf_Beta_FP1  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'FP1']
Inatdf_Beta_FP2  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'FP2']
Inatdf_Beta_F3  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'F3']
Inatdf_Beta_F4  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'F4']
Inatdf_Beta_Fz  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'Fz']
Inatdf_Beta_C3  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'C3']
Inatdf_Beta_C4  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'C4']
Inatdf_Beta_Cz  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'Cz']
Inatdf_Beta_P3  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'P3']
Inatdf_Beta_P4  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'P4']
Inatdf_Beta_Pz  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'Pz']
Inatdf_Beta_O1  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'O1']
Inatdf_Beta_O2  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'O2']
Inatdf_Beta_F7  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'F7']
Inatdf_Beta_F8  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'F8']
Inatdf_Beta_T3  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'T3']
Inatdf_Beta_T4  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'T4']
Inatdf_Beta_T5  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'T5']
Inatdf_Beta_T6  = df_Beta_inat.loc[df_Beta_inat['electrode'] == 'T6']

#Make a df for each electrode for mixed in beta
Mixdf_Beta_FP1  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'FP1']
Mixdf_Beta_FP2  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'FP2']
Mixdf_Beta_F3  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'F3']
Mixdf_Beta_F4  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'F4']
Mixdf_Beta_Fz  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'Fz']
Mixdf_Beta_C3  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'C3']
Mixdf_Beta_C4  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'C4']
Mixdf_Beta_Cz  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'Cz']
Mixdf_Beta_P3  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'P3']
Mixdf_Beta_P4  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'P4']
Mixdf_Beta_Pz  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'Pz']
Mixdf_Beta_O1  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'O1']
Mixdf_Beta_O2  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'O2']
Mixdf_Beta_F7  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'F7']
Mixdf_Beta_F8  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'F8']
Mixdf_Beta_T3  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'T3']
Mixdf_Beta_T4  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'T4']
Mixdf_Beta_T5  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'T5']
Mixdf_Beta_T6  = df_Beta_mixed.loc[df_Beta_mixed['electrode'] == 'T6']



## Get DELTA p values


In [23]:
# prepare variable for pvals for future mask
ipvalsD = []

Inatdf_Delta_C3 = Inatdf_Delta_C3[['fft_abs_power']]
Mixdf_Delta_C3 = Mixdf_Delta_C3[['fft_abs_power']]
data1 = Inatdf_Delta_C3
data2 = Mixdf_Delta_C3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values


Inatdf_Delta_C4 = Inatdf_Delta_C4[['fft_abs_power']]
Mixdf_Delta_C4 = Mixdf_Delta_C4[['fft_abs_power']]
data1 = Inatdf_Delta_C4
data2 = Mixdf_Delta_C4


# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values


Inatdf_Delta_Cz = Inatdf_Delta_Cz[['fft_abs_power']]
Mixdf_Delta_Cz = Mixdf_Delta_Cz[['fft_abs_power']]
data1 = Inatdf_Delta_Cz
data2 = Mixdf_Delta_Cz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values


Inatdf_Delta_F3 = Inatdf_Delta_F3[['fft_abs_power']]
Mixdf_Delta_F3 = Mixdf_Delta_F3[['fft_abs_power']]
data1 = Inatdf_Delta_F3
data2 = Mixdf_Delta_F3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values


Inatdf_Delta_F4 = Inatdf_Delta_F4[['fft_abs_power']]
Mixdf_Delta_F4 = Mixdf_Delta_F4[['fft_abs_power']]
data1 = Inatdf_Delta_F4
data2 = Mixdf_Delta_F4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values


Inatdf_Delta_F7 = Inatdf_Delta_F7[['fft_abs_power']]
Mixdf_Delta_F7 = Mixdf_Delta_F7[['fft_abs_power']]
data1 = Inatdf_Delta_F7
data2 = Mixdf_Delta_F7

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values


Inatdf_Delta_F8 = Inatdf_Delta_F8[['fft_abs_power']]
Mixdf_Delta_F8 = Mixdf_Delta_F8[['fft_abs_power']]
data1 = Inatdf_Delta_F8
data2 = Mixdf_Delta_F8

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values


Inatdf_Delta_FP1 = Inatdf_Delta_FP1[['fft_abs_power']]
Mixdf_Delta_FP1 = Mixdf_Delta_FP1[['fft_abs_power']]
data1 = Inatdf_Delta_FP1
data2 = Mixdf_Delta_FP1

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values


Inatdf_Delta_FP2 = Inatdf_Delta_FP2[['fft_abs_power']]
Mixdf_Delta_FP2 = Mixdf_Delta_FP2[['fft_abs_power']]
data1 = Inatdf_Delta_FP2
data2 = Mixdf_Delta_FP2

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values


Inatdf_Delta_Fz = Inatdf_Delta_Fz[['fft_abs_power']]
Mixdf_Delta_Fz = Mixdf_Delta_Fz[['fft_abs_power']]
data1 = Inatdf_Delta_Fz
data2 = Mixdf_Delta_Fz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values


Inatdf_Delta_O1 = Inatdf_Delta_O1[['fft_abs_power']]
Mixdf_Delta_O1 = Mixdf_Delta_O1[['fft_abs_power']]
data1 = Inatdf_Delta_O1
data2 = Mixdf_Delta_O1

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values


Inatdf_Delta_O2 = Inatdf_Delta_O2[['fft_abs_power']]
Mixdf_Delta_O2 = Mixdf_Delta_O2[['fft_abs_power']]
data1 = Inatdf_Delta_O2
data2 = Mixdf_Delta_O2

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values


Inatdf_Delta_P3 = Inatdf_Delta_P3[['fft_abs_power']]
Mixdf_Delta_P3 = Mixdf_Delta_P3[['fft_abs_power']]
data1 = Inatdf_Delta_P3
data2 = Mixdf_Delta_P3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values


Inatdf_Delta_P4 = Inatdf_Delta_P4[['fft_abs_power']]
Mixdf_Delta_P4 = Mixdf_Delta_P4[['fft_abs_power']]
data1 = Inatdf_Delta_P4
data2 = Mixdf_Delta_P4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values


Inatdf_Delta_Pz = Inatdf_Delta_Pz[['fft_abs_power']]
Mixdf_Delta_Pz = Mixdf_Delta_Pz[['fft_abs_power']]
data1 = Inatdf_Delta_Pz
data2 = Mixdf_Delta_Pz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values


Inatdf_Delta_T3 = Inatdf_Delta_T3[['fft_abs_power']]
Mixdf_Delta_T3 = Mixdf_Delta_T3[['fft_abs_power']]
data1 = Inatdf_Delta_T3
data2 = Mixdf_Delta_T3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values


Inatdf_Delta_T4 = Inatdf_Delta_T4[['fft_abs_power']]
Mixdf_Delta_T4 = Mixdf_Delta_T4[['fft_abs_power']]
data1 = Inatdf_Delta_T4
data2 = Mixdf_Delta_T4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values


Inatdf_Delta_T5 = Inatdf_Delta_T5[['fft_abs_power']]
Mixdf_Delta_T5 = Mixdf_Delta_T5[['fft_abs_power']]
data1 = Inatdf_Delta_T5
data2 = Mixdf_Delta_T5

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values


Inatdf_Delta_T6 = Inatdf_Delta_T6[['fft_abs_power']]
Mixdf_Delta_T6 = Mixdf_Delta_T6[['fft_abs_power']]
data1 = Inatdf_Delta_T6
data2 = Mixdf_Delta_T6

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsD.append(p) #truc pour updater l'array des p-values
print(ipvalsD)


Statistics=870.000, p=0.039
Statistics=836.000, p=0.022
Statistics=910.000, p=0.072
Statistics=863.000, p=0.034
Statistics=867.000, p=0.037
Statistics=734.000, p=0.003
Statistics=577.000, p=0.000
Statistics=813.000, p=0.014
Statistics=760.000, p=0.005
Statistics=938.000, p=0.105
Statistics=822.000, p=0.017
Statistics=937.000, p=0.104
Statistics=858.000, p=0.032
Statistics=906.000, p=0.068
Statistics=889.000, p=0.052
Statistics=795.000, p=0.010
Statistics=814.000, p=0.014
Statistics=795.000, p=0.010
Statistics=944.000, p=0.114
[0.03868923653161685, 0.021522843850494335, 0.07165723151335743, 0.034451987395849246, 0.036823965140120145, 0.0025969537189734888, 3.4134090555467823e-05, 0.014002029632473595, 0.004687046079748884, 0.1053254004466243, 0.01662063691535879, 0.10395350037547296, 0.0316650440792494, 0.0676121833706957, 0.05235940626639477, 0.009814248255139567, 0.014274238149001077, 0.009814248255139567, 0.11383237348324493]


In [24]:
pvals = ipvalsD

# Create a list of the adjusted p-values
p_adjusted_D = multipletests(pvals, alpha=0.05, method='bonferroni')

# Print the resulting conclusions
print(p_adjusted_D)
p_adjusted_D = p_adjusted_D[1]
p_adjusted_D

(array([False, False, False, False, False,  True,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False]), array([7.35095494e-01, 4.08934033e-01, 1.00000000e+00, 6.54587761e-01,
       6.99655338e-01, 4.93421207e-02, 6.48547721e-04, 2.66038563e-01,
       8.90538755e-02, 1.00000000e+00, 3.15792101e-01, 1.00000000e+00,
       6.01635838e-01, 1.00000000e+00, 9.94828719e-01, 1.86470717e-01,
       2.71210525e-01, 1.86470717e-01, 1.00000000e+00]), 0.0026960063028712566, 0.002631578947368421)


array([7.35095494e-01, 4.08934033e-01, 1.00000000e+00, 6.54587761e-01,
       6.99655338e-01, 4.93421207e-02, 6.48547721e-04, 2.66038563e-01,
       8.90538755e-02, 1.00000000e+00, 3.15792101e-01, 1.00000000e+00,
       6.01635838e-01, 1.00000000e+00, 9.94828719e-01, 1.86470717e-01,
       2.71210525e-01, 1.86470717e-01, 1.00000000e+00])

## Get THETA p values

In [25]:
# prepare variable for pvals for future mask
ipvalsT = []

Inatdf_Theta_C3 = Inatdf_Theta_C3[['fft_abs_power']]
Mixdf_Theta_C3 = Mixdf_Theta_C3[['fft_abs_power']]
data1 = Inatdf_Theta_C3
data2 = Mixdf_Theta_C3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values


Inatdf_Theta_C4 = Inatdf_Theta_C4[['fft_abs_power']]
Mixdf_Theta_C4 = Mixdf_Theta_C4[['fft_abs_power']]
data1 = Inatdf_Theta_C4
data2 = Mixdf_Theta_C4


# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values


Inatdf_Theta_Cz = Inatdf_Theta_Cz[['fft_abs_power']]
Mixdf_Theta_Cz = Mixdf_Theta_Cz[['fft_abs_power']]
data1 = Inatdf_Theta_Cz
data2 = Mixdf_Theta_Cz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values


Inatdf_Theta_F3 = Inatdf_Theta_F3[['fft_abs_power']]
Mixdf_Theta_F3 = Mixdf_Theta_F3[['fft_abs_power']]
data1 = Inatdf_Theta_F3
data2 = Mixdf_Theta_F3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values


Inatdf_Theta_F4 = Inatdf_Theta_F4[['fft_abs_power']]
Mixdf_Theta_F4 = Mixdf_Theta_F4[['fft_abs_power']]
data1 = Inatdf_Theta_F4
data2 = Mixdf_Theta_F4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values


Inatdf_Theta_F7 = Inatdf_Theta_F7[['fft_abs_power']]
Mixdf_Theta_F7 = Mixdf_Theta_F7[['fft_abs_power']]
data1 = Inatdf_Theta_F7
data2 = Mixdf_Theta_F7

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values


Inatdf_Theta_F8 = Inatdf_Theta_F8[['fft_abs_power']]
Mixdf_Theta_F8 = Mixdf_Theta_F8[['fft_abs_power']]
data1 = Inatdf_Theta_F8
data2 = Mixdf_Theta_F8

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values


Inatdf_Theta_FP1 = Inatdf_Theta_FP1[['fft_abs_power']]
Mixdf_Theta_FP1 = Mixdf_Theta_FP1[['fft_abs_power']]
data1 = Inatdf_Theta_FP1
data2 = Mixdf_Theta_FP1

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values


Inatdf_Theta_FP2 = Inatdf_Theta_FP2[['fft_abs_power']]
Mixdf_Theta_FP2 = Mixdf_Theta_FP2[['fft_abs_power']]
data1 = Inatdf_Theta_FP2
data2 = Mixdf_Theta_FP2

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values


Inatdf_Theta_Fz = Inatdf_Theta_Fz[['fft_abs_power']]
Mixdf_Theta_Fz = Mixdf_Theta_Fz[['fft_abs_power']]
data1 = Inatdf_Theta_Fz
data2 = Mixdf_Theta_Fz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values


Inatdf_Theta_O1 = Inatdf_Theta_O1[['fft_abs_power']]
Mixdf_Theta_O1 = Mixdf_Theta_O1[['fft_abs_power']]
data1 = Inatdf_Theta_O1
data2 = Mixdf_Theta_O1

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values


Inatdf_Theta_O2 = Inatdf_Theta_O2[['fft_abs_power']]
Mixdf_Theta_O2 = Mixdf_Theta_O2[['fft_abs_power']]
data1 = Inatdf_Theta_O2
data2 = Mixdf_Theta_O2

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values


Inatdf_Theta_P3 = Inatdf_Theta_P3[['fft_abs_power']]
Mixdf_Theta_P3 = Mixdf_Theta_P3[['fft_abs_power']]
data1 = Inatdf_Theta_P3
data2 = Mixdf_Theta_P3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values


Inatdf_Theta_P4 = Inatdf_Theta_P4[['fft_abs_power']]
Mixdf_Theta_P4 = Mixdf_Theta_P4[['fft_abs_power']]
data1 = Inatdf_Theta_P4
data2 = Mixdf_Theta_P4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values


Inatdf_Theta_Pz = Inatdf_Theta_Pz[['fft_abs_power']]
Mixdf_Theta_Pz = Mixdf_Theta_Pz[['fft_abs_power']]
data1 = Inatdf_Theta_Pz
data2 = Mixdf_Theta_Pz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values


Inatdf_Theta_T3 = Inatdf_Theta_T3[['fft_abs_power']]
Mixdf_Theta_T3 = Mixdf_Theta_T3[['fft_abs_power']]
data1 = Inatdf_Theta_T3
data2 = Mixdf_Theta_T3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values


Inatdf_Theta_T4 = Inatdf_Theta_T4[['fft_abs_power']]
Mixdf_Theta_T4 = Mixdf_Theta_T4[['fft_abs_power']]
data1 = Inatdf_Theta_T4
data2 = Mixdf_Theta_T4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values


Inatdf_Theta_T5 = Inatdf_Theta_T5[['fft_abs_power']]
Mixdf_Theta_T5 = Mixdf_Theta_T5[['fft_abs_power']]
data1 = Inatdf_Theta_T5
data2 = Mixdf_Theta_T5

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values


Inatdf_Theta_T6 = Inatdf_Theta_T6[['fft_abs_power']]
Mixdf_Theta_T6 = Mixdf_Theta_T6[['fft_abs_power']]
data1 = Inatdf_Theta_T6
data2 = Mixdf_Theta_T6

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsT.append(p) #truc pour updater l'array des p-values
print(ipvalsT)


Statistics=954.000, p=0.129
Statistics=999.000, p=0.215
Statistics=1043.000, p=0.324
Statistics=986.000, p=0.187
Statistics=1028.000, p=0.284
Statistics=895.000, p=0.057
Statistics=964.000, p=0.146
Statistics=940.000, p=0.108
Statistics=955.000, p=0.131
Statistics=1045.000, p=0.329
Statistics=923.000, p=0.086
Statistics=937.000, p=0.104
Statistics=929.000, p=0.093
Statistics=952.000, p=0.126
Statistics=965.000, p=0.147
Statistics=869.000, p=0.038
Statistics=983.000, p=0.181
Statistics=890.000, p=0.053
Statistics=977.000, p=0.169
[0.1290780373115687, 0.21464782112527742, 0.32362043481709235, 0.18707558049536172, 0.2839819385461839, 0.057396236955850125, 0.14568431301957185, 0.1081083969226922, 0.13067702545555948, 0.32907408021737433, 0.08609005563997074, 0.10395350037547296, 0.09344248079020923, 0.1259208865106165, 0.1474206027351223, 0.038059108109198596, 0.18103828764906577, 0.053173613831973686, 0.16933485980803165]


In [26]:
pvals = ipvalsT

# Create a list of the adjusted p-values
p_adjusted_T = multipletests(pvals, alpha=0.05, method='bonferroni')

# Print the resulting conclusions
print(p_adjusted_T)
p_adjusted_T = p_adjusted_T[1]
p_adjusted_T

(array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False]), array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       0.72312305, 1.        , 1.        , 1.        ]), 0.0026960063028712566, 0.002631578947368421)


array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       0.72312305, 1.        , 1.        , 1.        ])

## Get ALPHA p values

In [27]:
# prepare variable for pvals for future mask
ipvalsA = []

Inatdf_Alpha_C3 = Inatdf_Alpha_C3[['fft_abs_power']]
Mixdf_Alpha_C3 = Mixdf_Alpha_C3[['fft_abs_power']]
data1 = Inatdf_Alpha_C3
data2 = Mixdf_Alpha_C3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values


Inatdf_Alpha_C4 = Inatdf_Alpha_C4[['fft_abs_power']]
Mixdf_Alpha_C4 = Mixdf_Alpha_C4[['fft_abs_power']]
data1 = Inatdf_Alpha_C4
data2 = Mixdf_Alpha_C4


# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values


Inatdf_Alpha_Cz = Inatdf_Alpha_Cz[['fft_abs_power']]
Mixdf_Alpha_Cz = Mixdf_Alpha_Cz[['fft_abs_power']]
data1 = Inatdf_Alpha_Cz
data2 = Mixdf_Alpha_Cz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values


Inatdf_Alpha_F3 = Inatdf_Alpha_F3[['fft_abs_power']]
Mixdf_Alpha_F3 = Mixdf_Alpha_F3[['fft_abs_power']]
data1 = Inatdf_Alpha_F3
data2 = Mixdf_Alpha_F3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values


Inatdf_Alpha_F4 = Inatdf_Alpha_F4[['fft_abs_power']]
Mixdf_Alpha_F4 = Mixdf_Alpha_F4[['fft_abs_power']]
data1 = Inatdf_Alpha_F4
data2 = Mixdf_Alpha_F4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values


Inatdf_Alpha_F7 = Inatdf_Alpha_F7[['fft_abs_power']]
Mixdf_Alpha_F7 = Mixdf_Alpha_F7[['fft_abs_power']]
data1 = Inatdf_Alpha_F7
data2 = Mixdf_Alpha_F7

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values


Inatdf_Alpha_F8 = Inatdf_Alpha_F8[['fft_abs_power']]
Mixdf_Alpha_F8 = Mixdf_Alpha_F8[['fft_abs_power']]
data1 = Inatdf_Alpha_F8
data2 = Mixdf_Alpha_F8

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values


Inatdf_Alpha_FP1 = Inatdf_Alpha_FP1[['fft_abs_power']]
Mixdf_Alpha_FP1 = Mixdf_Alpha_FP1[['fft_abs_power']]
data1 = Inatdf_Alpha_FP1
data2 = Mixdf_Alpha_FP1

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values


Inatdf_Alpha_FP2 = Inatdf_Alpha_FP2[['fft_abs_power']]
Mixdf_Alpha_FP2 = Mixdf_Alpha_FP2[['fft_abs_power']]
data1 = Inatdf_Alpha_FP2
data2 = Mixdf_Alpha_FP2

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values


Inatdf_Alpha_Fz = Inatdf_Alpha_Fz[['fft_abs_power']]
Mixdf_Alpha_Fz = Mixdf_Alpha_Fz[['fft_abs_power']]
data1 = Inatdf_Alpha_Fz
data2 = Mixdf_Alpha_Fz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values


Inatdf_Alpha_O1 = Inatdf_Alpha_O1[['fft_abs_power']]
Mixdf_Alpha_O1 = Mixdf_Alpha_O1[['fft_abs_power']]
data1 = Inatdf_Alpha_O1
data2 = Mixdf_Alpha_O1

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values


Inatdf_Alpha_O2 = Inatdf_Alpha_O2[['fft_abs_power']]
Mixdf_Alpha_O2 = Mixdf_Alpha_O2[['fft_abs_power']]
data1 = Inatdf_Alpha_O2
data2 = Mixdf_Alpha_O2

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values


Inatdf_Alpha_P3 = Inatdf_Alpha_P3[['fft_abs_power']]
Mixdf_Alpha_P3 = Mixdf_Alpha_P3[['fft_abs_power']]
data1 = Inatdf_Alpha_P3
data2 = Mixdf_Alpha_P3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values


Inatdf_Alpha_P4 = Inatdf_Alpha_P4[['fft_abs_power']]
Mixdf_Alpha_P4 = Mixdf_Alpha_P4[['fft_abs_power']]
data1 = Inatdf_Alpha_P4
data2 = Mixdf_Alpha_P4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values


Inatdf_Alpha_Pz = Inatdf_Alpha_Pz[['fft_abs_power']]
Mixdf_Alpha_Pz = Mixdf_Alpha_Pz[['fft_abs_power']]
data1 = Inatdf_Alpha_Pz
data2 = Mixdf_Alpha_Pz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values


Inatdf_Alpha_T3 = Inatdf_Alpha_T3[['fft_abs_power']]
Mixdf_Alpha_T3 = Mixdf_Alpha_T3[['fft_abs_power']]
data1 = Inatdf_Alpha_T3
data2 = Mixdf_Alpha_T3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values


Inatdf_Alpha_T4 = Inatdf_Alpha_T4[['fft_abs_power']]
Mixdf_Alpha_T4 = Mixdf_Alpha_T4[['fft_abs_power']]
data1 = Inatdf_Alpha_T4
data2 = Mixdf_Alpha_T4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values


Inatdf_Alpha_T5 = Inatdf_Alpha_T5[['fft_abs_power']]
Mixdf_Alpha_T5 = Mixdf_Alpha_T5[['fft_abs_power']]
data1 = Inatdf_Alpha_T5
data2 = Mixdf_Alpha_T5

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values


Inatdf_Alpha_T6 = Inatdf_Alpha_T6[['fft_abs_power']]
Mixdf_Alpha_T6 = Mixdf_Alpha_T6[['fft_abs_power']]
data1 = Inatdf_Alpha_T6
data2 = Mixdf_Alpha_T6

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsA.append(p) #truc pour updater l'array des p-values
print(ipvalsA)


Statistics=834.000, p=0.021
Statistics=816.000, p=0.015
Statistics=847.000, p=0.026
Statistics=796.000, p=0.010
Statistics=813.000, p=0.014
Statistics=785.000, p=0.008
Statistics=799.000, p=0.011
Statistics=789.000, p=0.009
Statistics=806.000, p=0.012
Statistics=821.000, p=0.016
Statistics=840.000, p=0.023
Statistics=810.000, p=0.013
Statistics=800.000, p=0.011
Statistics=800.000, p=0.011
Statistics=816.000, p=0.015
Statistics=825.000, p=0.018
Statistics=788.000, p=0.009
Statistics=780.000, p=0.007
Statistics=831.000, p=0.020
[0.020755290973909168, 0.014832352327488809, 0.02618579380686838, 0.010014319816324597, 0.014002029632473595, 0.007997915286747754, 0.010636043753998627, 0.008685580086347946, 0.012218925776663276, 0.0163103615427259, 0.023130068602780097, 0.013212087593762942, 0.010850626978890101, 0.010850626978890101, 0.014832352327488809, 0.017581940076201737, 0.008508971988687691, 0.007205965303332377, 0.01964749073259759]


In [28]:
pvals = ipvalsA

# Create a list of the adjusted p-values
p_adjusted_A = multipletests(pvals, alpha=0.05, method='bonferroni')

# Print the resulting conclusions
print(p_adjusted_A)
p_adjusted_A = p_adjusted_A[1]
p_adjusted_A

(array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False]), array([0.39435053, 0.28181469, 0.49753008, 0.19027208, 0.26603856,
       0.15196039, 0.20208483, 0.16502602, 0.23215959, 0.30989687,
       0.4394713 , 0.25102966, 0.20616191, 0.20616191, 0.28181469,
       0.33405686, 0.16167047, 0.13691334, 0.37330232]), 0.0026960063028712566, 0.002631578947368421)


array([0.39435053, 0.28181469, 0.49753008, 0.19027208, 0.26603856,
       0.15196039, 0.20208483, 0.16502602, 0.23215959, 0.30989687,
       0.4394713 , 0.25102966, 0.20616191, 0.20616191, 0.28181469,
       0.33405686, 0.16167047, 0.13691334, 0.37330232])

## Get BETA p values

In [29]:
# prepare variable for pvals for future mask
ipvalsB = []

Inatdf_Beta_C3 = Inatdf_Beta_C3[['fft_abs_power']]
Mixdf_Beta_C3 = Mixdf_Beta_C3[['fft_abs_power']]
data1 = Inatdf_Beta_C3
data2 = Mixdf_Beta_C3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values


Inatdf_Beta_C4 = Inatdf_Beta_C4[['fft_abs_power']]
Mixdf_Beta_C4 = Mixdf_Beta_C4[['fft_abs_power']]
data1 = Inatdf_Beta_C4
data2 = Mixdf_Beta_C4


# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values


Inatdf_Beta_Cz = Inatdf_Beta_Cz[['fft_abs_power']]
Mixdf_Beta_Cz = Mixdf_Beta_Cz[['fft_abs_power']]
data1 = Inatdf_Beta_Cz
data2 = Mixdf_Beta_Cz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values


Inatdf_Beta_F3 = Inatdf_Beta_F3[['fft_abs_power']]
Mixdf_Beta_F3 = Mixdf_Beta_F3[['fft_abs_power']]
data1 = Inatdf_Beta_F3
data2 = Mixdf_Beta_F3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values


Inatdf_Beta_F4 = Inatdf_Beta_F4[['fft_abs_power']]
Mixdf_Beta_F4 = Mixdf_Beta_F4[['fft_abs_power']]
data1 = Inatdf_Beta_F4
data2 = Mixdf_Beta_F4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values


Inatdf_Beta_F7 = Inatdf_Beta_F7[['fft_abs_power']]
Mixdf_Beta_F7 = Mixdf_Beta_F7[['fft_abs_power']]
data1 = Inatdf_Beta_F7
data2 = Mixdf_Beta_F7

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values


Inatdf_Beta_F8 = Inatdf_Beta_F8[['fft_abs_power']]
Mixdf_Beta_F8 = Mixdf_Beta_F8[['fft_abs_power']]
data1 = Inatdf_Beta_F8
data2 = Mixdf_Beta_F8

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values


Inatdf_Beta_FP1 = Inatdf_Beta_FP1[['fft_abs_power']]
Mixdf_Beta_FP1 = Mixdf_Beta_FP1[['fft_abs_power']]
data1 = Inatdf_Beta_FP1
data2 = Mixdf_Beta_FP1

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values


Inatdf_Beta_FP2 = Inatdf_Beta_FP2[['fft_abs_power']]
Mixdf_Beta_FP2 = Mixdf_Beta_FP2[['fft_abs_power']]
data1 = Inatdf_Beta_FP2
data2 = Mixdf_Beta_FP2

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values


Inatdf_Beta_Fz = Inatdf_Beta_Fz[['fft_abs_power']]
Mixdf_Beta_Fz = Mixdf_Beta_Fz[['fft_abs_power']]
data1 = Inatdf_Beta_Fz
data2 = Mixdf_Beta_Fz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values


Inatdf_Beta_O1 = Inatdf_Beta_O1[['fft_abs_power']]
Mixdf_Beta_O1 = Mixdf_Beta_O1[['fft_abs_power']]
data1 = Inatdf_Beta_O1
data2 = Mixdf_Beta_O1

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values


Inatdf_Beta_O2 = Inatdf_Beta_O2[['fft_abs_power']]
Mixdf_Beta_O2 = Mixdf_Beta_O2[['fft_abs_power']]
data1 = Inatdf_Beta_O2
data2 = Mixdf_Beta_O2

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values


Inatdf_Beta_P3 = Inatdf_Beta_P3[['fft_abs_power']]
Mixdf_Beta_P3 = Mixdf_Beta_P3[['fft_abs_power']]
data1 = Inatdf_Beta_P3
data2 = Mixdf_Beta_P3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values


Inatdf_Beta_P4 = Inatdf_Beta_P4[['fft_abs_power']]
Mixdf_Beta_P4 = Mixdf_Beta_P4[['fft_abs_power']]
data1 = Inatdf_Beta_P4
data2 = Mixdf_Beta_P4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values


Inatdf_Beta_Pz = Inatdf_Beta_Pz[['fft_abs_power']]
Mixdf_Beta_Pz = Mixdf_Beta_Pz[['fft_abs_power']]
data1 = Inatdf_Beta_Pz
data2 = Mixdf_Beta_Pz

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values


Inatdf_Beta_T3 = Inatdf_Beta_T3[['fft_abs_power']]
Mixdf_Beta_T3 = Mixdf_Beta_T3[['fft_abs_power']]
data1 = Inatdf_Beta_T3
data2 = Mixdf_Beta_T3

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values


Inatdf_Beta_T4 = Inatdf_Beta_T4[['fft_abs_power']]
Mixdf_Beta_T4 = Mixdf_Beta_T4[['fft_abs_power']]
data1 = Inatdf_Beta_T4
data2 = Mixdf_Beta_T4

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values


Inatdf_Beta_T5 = Inatdf_Beta_T5[['fft_abs_power']]
Mixdf_Beta_T5 = Mixdf_Beta_T5[['fft_abs_power']]
data1 = Inatdf_Beta_T5
data2 = Mixdf_Beta_T5

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values


Inatdf_Beta_T6 = Inatdf_Beta_T6[['fft_abs_power']]
Mixdf_Beta_T6 = Mixdf_Beta_T6[['fft_abs_power']]
data1 = Inatdf_Beta_T6
data2 = Mixdf_Beta_T6

# Mann-Whitney U test
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
ipvalsB.append(p) #truc pour updater l'array des p-values
print(ipvalsB)


Statistics=703.000, p=0.001
Statistics=717.000, p=0.002
Statistics=738.000, p=0.003
Statistics=742.000, p=0.003
Statistics=807.000, p=0.012
Statistics=756.000, p=0.004
Statistics=802.000, p=0.011
Statistics=869.000, p=0.038
Statistics=870.000, p=0.039
Statistics=751.000, p=0.004
Statistics=815.000, p=0.015
Statistics=759.000, p=0.005
Statistics=723.000, p=0.002
Statistics=669.000, p=0.001
Statistics=701.000, p=0.001
Statistics=1103.000, p=0.498
Statistics=922.000, p=0.085
Statistics=772.000, p=0.006
Statistics=838.000, p=0.022
[0.001225902278547461, 0.0017315111504776362, 0.0028505013056513645, 0.003126165541731463, 0.012460949224382846, 0.004289940767818088, 0.011291121318288376, 0.038059108109198596, 0.03868923653161685, 0.00383600440812039, 0.014550992441935525, 0.004584810226764927, 0.0020013133428987157, 0.0005074024627044765, 0.0011659023763096856, 0.49849128113498475, 0.08490793147185222, 0.006082222772559617, 0.02231425150052057]


In [30]:
pvals = ipvalsB

# Create a list of the adjusted p-values
p_adjusted_B = multipletests(pvals, alpha=0.05, method='bonferroni')

# Print the resulting conclusions
print(p_adjusted_B)
p_adjusted_B = p_adjusted_B[1]
p_adjusted_B

(array([ True,  True, False, False, False, False, False, False, False,
       False, False, False,  True,  True,  True, False, False, False,
       False]), array([0.02329214, 0.03289871, 0.05415952, 0.05939715, 0.23675804,
       0.08150887, 0.21453131, 0.72312305, 0.73509549, 0.07288408,
       0.27646886, 0.08711139, 0.03802495, 0.00964065, 0.02215215,
       1.        , 1.        , 0.11556223, 0.42397078]), 0.0026960063028712566, 0.002631578947368421)


array([0.02329214, 0.03289871, 0.05415952, 0.05939715, 0.23675804,
       0.08150887, 0.21453131, 0.72312305, 0.73509549, 0.07288408,
       0.27646886, 0.08711139, 0.03802495, 0.00964065, 0.02215215,
       1.        , 1.        , 0.11556223, 0.42397078])

In [33]:
pvals_adjusted_D = pd.DataFrame(p_adjusted_D) 
pvals_adjusted_T = pd.DataFrame(p_adjusted_T) 
pvals_adjusted_A = pd.DataFrame(p_adjusted_A) 
pvals_adjusted_B = pd.DataFrame(p_adjusted_B) 
subtype = pd.concat([pvals_adjusted_D, pvals_adjusted_T, pvals_adjusted_A, pvals_adjusted_B])
#gender_pvals = np.asarray(pvalsss)
subtype.to_csv('Data/subtype_pvals')